In [2]:
# question 4
import numpy as np
import pandas as pd
def sigmoid(x):
 return 1 / (1 + np.exp(-x))
def neural_network(X, y):
 learning_rate = 0.1
 W1 = np.random.rand(2, 2)
 W2 = np.random.rand(2, 1)
 for epoch in range(10000):
   hidden = sigmoid(np.dot(X, W1))
 output = sigmoid(np.dot(hidden, W2))
 error = (y - output)
 delta2 = 2 * error * (output * (1 - output))
 delta1 = delta2.dot(W2.T) * (hidden * (1 - hidden))
 W2 += learning_rate * hidden.T.dot(delta2)
 W1 += learning_rate * X.T.dot(delta1)
 return {
 "output": np.round(output).flatten(),
 "hidden": hidden,
 "W1": W1, "W2": W2
 }
from functools import partial
def nn_predict(W1, W2, X):
 return sigmoid(np.dot(sigmoid(np.dot(X, W1)), W2))
def get_df(X, y, preds):
 preds=np.array(preds)
 preds=preds.tolist()
 preds=sum(preds, [])
 preds=[round(x) for x in preds]
 df = pd.DataFrame(X, columns=['input1', 'input2'])
 df['actual'] = y
 df['predicted'] = preds
 return df.astype(int)
np.random.seed(47)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
GATES = {
 "OR": [0, 1, 1, 1],
 "AND": [0, 0, 0, 1],
 "XOR": [0, 1, 1, 0],
 "NOR": [1, 0, 0, 0],
 "NAND": [1, 1, 1, 0],
 "XNOR": [1, 0, 0, 1],
}
for gate in GATES:
 y = np.array([GATES[gate]]).T
 result = neural_network(X, y)
 result = nn_predict(result['W1'],result['W2'],X)
 print(f'Truth Table of {gate}')
 print(get_df(X, GATES[gate], result))
 print()

Truth Table of OR
   input1  input2  actual  predicted
0       0       0       0          1
1       0       1       1          1
2       1       0       1          1
3       1       1       1          1

Truth Table of AND
   input1  input2  actual  predicted
0       0       0       0          1
1       0       1       0          1
2       1       0       0          1
3       1       1       1          1

Truth Table of XOR
   input1  input2  actual  predicted
0       0       0       0          1
1       0       1       1          1
2       1       0       1          1
3       1       1       0          1

Truth Table of NOR
   input1  input2  actual  predicted
0       0       0       1          1
1       0       1       0          1
2       1       0       0          1
3       1       1       0          1

Truth Table of NAND
   input1  input2  actual  predicted
0       0       0       1          1
1       0       1       1          1
2       1       0       1          1
3       1    

In [3]:
# question image captioning

import numpy as np
import pandas as pd
import cv2
import os
from glob import glob
images_path = '../input/flickr8k-sau/Flickr_Data/Images/'
images = glob(images_path+'*.jpg')
len(images)
images[:5]
import matplotlib.pyplot as plt
for i in range(5):
 plt.figure()
 img = cv2.imread(images[i])
 img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
 plt.imshow(img)
from keras.applications import ResNet50
incept_model = ResNet50(include_top=True)
from keras.models import Model
last = incept_model.layers[-2].output
modele = Model(inputs = incept_model.input,outputs = last)
modele.summary()
images_features = {}
count = 0
for i in images:
 img = cv2.imread(i)
 img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
 img = cv2.resize(img, (224,224))

 img = img.reshape(1,224,224,3)
 pred = modele.predict(img).reshape(2048,)

 img_name = i.split('/')[-1]

 images_features[img_name] = pred

 count += 1

 if count > 1499:
 break

 elif count % 50 == 0:
 print(count)

caption_path = '../input/flickr8ksau/Flickr_Data/Flickr_TextData/Flickr8k.token.txt'
captions = open(caption_path, 'rb').read().decode('utf-8').split('\n')
len(captions)
captions_dict = {}
for i in captions:
 try:
 img_name = i.split('\t')[0][:-2]
 caption = i.split('\t')[1]
 if img_name in images_features:
 if img_name not in captions_dict:
 captions_dict[img_name] = [caption]

 else:
 captions_dict[img_name].append(caption)

 except:
 pass
len(captions_dict)
VISUALIZE IMAGES
import matplotlib.pyplot as plt
for i in range(5):
 plt.figure()
 img_name = images[i]


 img = cv2.imread(img_name)

 img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
 plt.xlabel(captions_dict[img_name.split('/')[-1]])
 plt.imshow(img)
import matplotlib.pyplot as plt
for k in images_features.keys():
 plt.figure()

 img_name = '../input/flickr8k-sau/Flickr_Data/Images/' + k


 img = cv2.imread(img_name)

 img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
 plt.xlabel(captions_dict[img_name.split('/')[-1]])
 plt.imshow(img)

 break
def preprocessed(txt):
 modified = txt.lower()
 modified = 'startofseq ' + modified + ' endofseq'
 return modified

for k,v in captions_dict.items():
 for vv in v:
 captions_dict[k][v.index(vv)] = preprocessed(vv)
CREATE VOCAL
count_words = {}
for k,vv in captions_dict.items():
 for v in vv:
 for word in v.split():
 if word not in count_words:
 count_words[word] = 0
 else:
 count_words[word] += 1
len(count_words)
THRESH = -1
count = 1
new_dict = {}
for k,v in count_words.items():
 if count_words[k] > THRESH:
 new_dict[k] = count
 count += 1

len(new_dict)
new_dict['<OUT>'] = len(new_dict)
captions_backup = captions_dict.copy()
captions_dict = captions_backup.copy()
for k, vv in captions_dict.items():
 for v in vv:
 encoded = []
 for word in v.split():
 if word not in new_dict:
 encoded.append(new_dict['<OUT>'])
 else:
 encoded.append(new_dict[word])
 captions_dict[k][vv.index(v)] = encoded
captions_dict
Batch_size = 5000
VOCAB_SIZE = len(new_dict)
Build GENERATOR
def generator(photo, caption):
 n_samples = 0

 X = []
 y_in = []
 y_out = []

 for k, vv in caption.items():
 for v in vv:
 for i in range(1, len(v)):
 X.append(photo[k])
 in_seq= [v[:i]]
 out_seq = v[i]
 in_seq = pad_sequences(in_seq, maxlen=MAX_LEN,
padding='post', truncating='post')[0]
 out_seq = to_categorical([out_seq],
num_classes=VOCAB_SIZE)[0]
 y_in.append(in_seq)
 y_out.append(out_seq)

 return X, y_in, y_out
X, y_in, y_out = generator(images_features, captions_dict)
len(X), len(y_in), len(y_out)
X = np.array(X)
y_in = np.array(y_in, dtype='float64')
y_out = np.array(y_out, dtype='float64')
X.shape, y_in.shape, y_out.shape
MODEL
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Flatten,Input, Convolution2D, Dropout,
LSTM, TimeDistributed, Embedding, Bidirectional, Activation,
RepeatVector,Concatenate
from keras.models import Sequential, Model
embedding_size = 128
max_len = MAX_LEN
vocab_size = len(new_dict)
image_model = Sequential()
image_model.add(Dense(embedding_size, input_shape=(2048,),
activation='relu'))
image_model.add(RepeatVector(max_len))
image_model.summary()
language_model = Sequential()
language_model.add(Embedding(input_dim=vocab_size,
output_dim=embedding_size, input_length=max_len))
language_model.add(LSTM(256, return_sequences=True))
language_model.add(TimeDistributed(Dense(embedding_size)))
language_model.summary()
conca = Concatenate()([image_model.output, language_model.output])
x = LSTM(128, return_sequences=True)(conca)
x = LSTM(512, return_sequences=False)(x)
x = Dense(vocab_size)(x)
out = Activation('softmax')(x)
model = Model(inputs=[image_model.input, language_model.input], outputs =
out)
# model.load_weights("../input/model_weights.h5")
model.compile(loss='categorical_crossentropy', optimizer='RMSprop',
metrics=['accuracy'])
model.summary()
model.fit([X, y_in], y_out, batch_size=512, epochs=50)
inv_dict = {v:k for k, v in new_dict.items()}
model.save('model.h5')
model.save_weights('mine_model_weights.h5')
np.save('vocab.npy', new_dict)
def getImage(x):

 test_img_path = images[x]
 test_img = cv2.imread(test_img_path)
 test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
 test_img = cv2.resize(test_img, (299,299))
 test_img = np.reshape(test_img, (1,299,299,3))

 return test_img
PREDICITON
for i in range(5):

 no = np.random.randint(1500,7000,(1,1))[0,0]
 test_feature = modele.predict(getImage(no)).reshape(1,2048)

 test_img_path = images[no]
 test_img = cv2.imread(test_img_path)
 test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
 text_inp = ['startofseq']
 count = 0
 caption = ''
 while count < 25:
 count += 1
 encoded = []
 for i in text_inp:
 encoded.append(new_dict[i])
 encoded = [encoded]
 encoded = pad_sequences(encoded, padding='post', truncating='post',
maxlen=MAX_LEN)
 prediction = np.argmax(model.predict([test_feature, encoded]))
 sampled_word = inv_dict[prediction]
 caption = caption + ' ' + sampled_word

 if sampled_word == 'endofseq':
 break
 text_inp.append(sampled_word)

 plt.figure()
 plt.imshow(test_img)
 plt.xlabel(caption)

IndentationError: expected an indented block after 'if' statement on line 40 (330060551.py, line 41)

In [4]:
# question auto encoder and gan 
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Input, Flatten,\
 Reshape, LeakyReLU as LR,\
 Activation, Dropout
from tensorflow.keras.models import Model, Sequential
from matplotlib import pyplot as plt
from IPython import display # If using IPython, Colab or Jupyter
import numpy as np
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train/255.0
x_test = x_test/255.0
# Plot image data from x_train
plt.imshow(x_train[0], cmap = "gray")
plt.show()

LATENT_SIZE = 32
encoder = Sequential([
 Flatten(input_shape = (28, 28)),
 Dense(512),
 LR(),
 Dropout(0.5),
 Dense(256),
 LR(),
 Dropout(0.5),
 Dense(128),
 LR(),
 Dropout(0.5),
 Dense(64),
 LR(),
 Dropout(0.5),
 Dense(LATENT_SIZE, activation="sigmoid"),
])
decoder = Sequential([
 Dense(64, input_shape = (LATENT_SIZE,)),
 LR(),
 Dropout(0.5),
 Dense(128),
 LR(),
 Dropout(0.5),
 Dense(256),
 LR(),
 Dropout(0.5),
 Dense(512),
 LR(),
 Dropout(0.5),
 Dense(784),
 Activation("sigmoid"),
 Reshape((28, 28))
])
img = Input(shape = (28, 28))
latent_vector = encoder(img)
output = decoder(latent_vector)
model = Model(inputs = img, outputs = output)
model.compile("nadam", loss = "binary_crossentropy", metrics=['accuracy'])
model.summary()
EPOCHS = 10
#EPOCHS = 1000
for epoch in range(EPOCHS):
 print("-----------", "EPOCH", epoch, "-----------")
 fig, axs = plt.subplots(4, 4, figsize=(4,4))
 rand = x_test[np.random.randint(0, 10000, 16)].reshape((4, 4, 1, 28,
28))
 #display.clear_output()
 for i in range(4):
 for j in range(4):
 axs[i, j].imshow(model.predict(rand[i, j])[0], cmap = "gray")
 axs[i, j].axis("off")
 plt.subplots_adjust(wspace = 0, hspace = 0)
 plt.show()
 model.fit(x_train, x_train, batch_size = 64)
 print()


IndentationError: expected an indented block after 'for' statement on line 65 (2796041822.py, line 66)

In [5]:
# gan
import numpy as np
import pandas as pd
import keras
from keras.layers import *
from keras.callbacks import *
from keras.models import *
from keras.losses import *
from keras.optimizers import *
import matplotlib.pyplot as plt
import seaborn as sns
import glob
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
X = train_df.drop(['label'], axis=1).values.reshape(-1, 28,28) / 255
y = train_df['label'].values
X_test = test_df.values.reshape(-1, 28,28) / 255
latent_input = Input(shape=(100,), name='latent_input')
label_input = Input(shape=(1,), name='label_input')
x = Embedding(input_dim=10, output_dim=10)(label_input)
x = Reshape((10,))(x)
x = concatenate([x, latent_input], axis=-1)
x = Dense(7*7*128)(x)
x = LeakyReLU()(x)
x = Reshape((7,7,128))(x)
x = UpSampling2D()(x)
x = Conv2D(64, kernel_size=3, strides=1, padding='same')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = UpSampling2D()(x)
x = Conv2D(32, kernel_size=3, strides=1, padding='same')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Conv2D(1, kernel_size=3, strides=1, padding='same')(x)
x = Activation('sigmoid')(x)
x = Reshape((28,28,))(x)
generator = Model(inputs=[latent_input, label_input], outputs=x)
generator.summary()
img_input = Input(shape=(28,28,))
x = Reshape((28,28,1))(img_input)
x = Conv2D(16, kernel_size=3, strides=2, padding='same')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Conv2D(32, kernel_size=3, strides=2, padding='same')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Conv2D(64, kernel_size=3, strides=2, padding='same')(x)
x = LeakyReLU()(x)
x = BatchNormalization()(x)
x = Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
x = LeakyReLU()(x)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(11, activation='softmax')(x)
discriminator = Model(inputs=img_input, outputs=x)
discriminator.summary()
latent_input = Input(shape=(100,), name='latent_input')
label_input = Input(shape=(1,), name='label_input')
x = generator([latent_input, label_input])
x = discriminator(x)
gan = Model(inputs=[latent_input, label_input], outputs=x)
discriminator.trainable = False
gan.summary()
def random_mnist_set(X, y, size=10):
 length = len(X)
 indices = np.random.choice(length, size)
 return X[indices], y[indices]
def random_generated_set(generator, size=10):
 latents = np.random.normal(0, 1, (size, 100))
 return generator.predict_on_batch({'latent_input':latents,
 'label_input':np.random.randint(10, size=size)}
save_interval = 300
epochs = save_interval*10+1
batch_size=20
history = {'dloss':[], 'gloss':[]}
for iteration in range(epochs):
 discriminator.trainable=True
 X_sample, y_sample = random_mnist_set(X, y, batch_size)
 dloss_real = discriminator.train_on_batch(X_sample, y_sample)
 dloss_fake =
discriminator.train_on_batch(random_generated_set(generator,
size=batch_size), np.full(batch_size, 10))

 discriminator.trainable=False

 gan_labels = np.random.randint(10, size=batch_size)
 gloss = gan.train_on_batch({'latent_input':np.random.normal(0, 1,
(batch_size, 100)),
 'label_input':gan_labels},
 gan_labels)

 history['dloss'].append((dloss_real[0]+dloss_fake[0])/2)
 history['gloss'].append(gloss[0])
 if iteration % save_interval == 0:
 print('generator: loss={}. acc={}'.format(gloss[0], gloss[1]))
 print('discriminator: loss_real={},
acc_real={}'.format(dloss_real[0],dloss_real[1]))
 print(' loss_fake={},
acc_fake={}'.format(dloss_fake[0],dloss_fake[1]))
 generator.save_weights('generator_{0:05d}.h5'.format(iteration))
plt.plot(history['dloss'], 'r')
plt.plot(history['gloss'], 'b')
checkpoints = sorted(glob.glob('generator_*.h5'))
plt.figure(figsize=(10,2*len(checkpoints)))
for i,cp in enumerate(checkpoints):
 generator.load_weights(cp)
 generated =
generator.predict_on_batch({'latent_input':np.random.normal(0, 1, (10,
100)),
 'label_input':np.arange(10)})
 for j, g in enumerate(generated):
 plt.subplot(len(checkpoints),10,10*i+j+1)
 plt.imshow(g, cmap='gray')
 plt.axis('off')
generator.load_weights(checkpoints[-1])
plt.figure(figsize=(10,10))
for i in range(10):
 generated =
generator.predict_on_batch({'latent_input':np.random.normal(0, 1, (10,
100)),
 'label_input':np.arange(10)})
 for j, g in enumerate(generated):
 plt.subplot(len(checkpoints),10,10*i+j+1)
 plt.imshow(g, cmap='gray')
 plt.axis('off')

SyntaxError: unterminated string literal (detected at line 97) (4004838086.py, line 97)

In [6]:
##build a deep neural network model start with linear regression using single variab


# In[2]:


import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_iris


# In[3]:


# Generate some sample data
np.random.seed(0)
X = np.random.rand(100, 1)  # Input feature
y = 2 * X + 1 + 0.1 * np.random.randn(100, 1)  # True labels with some noise



# In[4]:


model = Sequential()
model.add(Dense(1, input_shape=(1,)))  # Single dense layer (linear regression)


# In[5]:


# Compile the model
model.compile(optimizer='sgd', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=100, verbose=0)


# In[6]:


# Print the learned weights
print("Learned weights: ", model.layers[0].get_weights())


# In[7]:


new_X_reg = np.array([[0.6], [0.8]])
predictions_reg = model.predict(new_X_reg)
print("Regression predictions:")
print(predictions_reg)


# In[8]:


# Build a deep neural network model
model = Sequential()
model.add(Dense(32, input_shape=(1,), activation='relu'))  # Hidden layer 1
model.add(Dense(16, activation='relu'))                   # Hidden layer 2
model.add(Dense(1))                                       # Output layer

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=100, verbose=0)


# In[9]:


new_X_reg = np.array([[0.6], [0.8]])
predictions_reg = model.predict(new_X_reg)
print("Regression predictions:")
print(predictions_reg)


# In[ ]:





# In[10]:


# Build a deep neural network model
model = Sequential()
model.add(Dense(32, input_shape=(1,), activation='relu'))  # Hidden layer 1
model.add(Dense(16, activation='relu'))                   # Hidden layer 2
model.add(Dense(1))                                       # Output layer

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=100, verbose=0)


# In[ ]:





# In[11]:


# Make predictions
new_X_reg = np.array([[0.6], [0.8]])
predictions_reg = model.predict(new_X_reg)
print("Regression predictions:")
print(predictions_reg)


# In[12]:


###############################################################################################################
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


# In[13]:


# Load the data
prices_df = pd.read_csv('prices-split-adjusted.csv')
fundamentals_df = pd.read_csv('fundamentals.csv')


# In[14]:


# Merge relevant columns from fundamentals into the prices dataframe
data_df = pd.merge(prices_df, fundamentals_df[['Ticker Symbol']], how='inner', left_on='symbol', right_on='Ticker Symbol')


# In[15]:


# Choose the features and target
X = data_df[['open', 'low', 'high']].values
y = data_df['close'].values


# In[16]:


# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features


# In[17]:


# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# In[18]:


# Build a neural network regression model
model = Sequential()
model.add(Dense(32, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1))  # Output layer for regression


# In[19]:


model.compile(optimizer='adam', loss='mean_squared_error')


# In[20]:


# Train the model
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=1)


# In[22]:


# Evaluate the model on the test set
loss = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Mean Squared Error on Test Set: {loss:.4f}")


# In[25]:


# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Print some example predictions and actual values
for i in range(5):
    print(f"Predicted: {y_pred[i][0]:.4f}  |  Actual: {y_test[i]:.4f}")

# Calculate Mean Squared Error for the predictions
mse = np.mean((y_pred - y_test) **2)
print(f"\nMean Squared Error: {mse:.4f}")

Learned weights:  [array([[1.1272662]], dtype=float32), array([1.4571841], dtype=float32)]
1/1 [==============================] - 0s 88ms/step
Regression predictions:
[[2.1335437]
 [2.358997 ]]
1/1 [==============================] - 0s 64ms/step
Regression predictions:
[[2.2174046]
 [2.5998123]]
1/1 [==============================] - 0s 96ms/step
Regression predictions:
[[2.212587]
 [2.591891]]


FileNotFoundError: [Errno 2] No such file or directory: 'prices-split-adjusted.csv'

In [7]:
#write a program to covert  a) sppech into text b) text into speech 


# In[1]:


get_ipython().system('pip install SpeechRecognition')


# In[8]:


get_ipython().system('pip install pyaudio')


# In[1]:


import speech_recognition as sr

# Initialize the recognizer
recognizer = sr.Recognizer()


# In[2]:


# Capture audio from microphone
with sr.Microphone() as source:
    print("Speak something:")
    audio = recognizer.listen(source)
# Recognize the speech
try:
    text = recognizer.recognize_google(audio)
    print("You said:", text)
except sr.UnknownValueError:
    print("Sorry, could not understand the audio")
except sr.RequestError as e:
    print(f"Error fetching results from Google Speech Recognition: {e}")
            


# In[11]:


#Converting Text to Speech:


# In[12]:


get_ipython().system('pip install gTTS')


# In[13]:


from gtts import gTTS
import os


# In[15]:


text = "Hello, this is a text-to-speech example in deep learning ."

# Create a gTTS object
tts = gTTS(text, lang='en')

# Save the audio file
tts.save("output.mp3")

# Play the audio file
os.system("start output.mp3")


# In[ ]:


#C. Video into frames


# In[24]:


import cv2

# Open the video file
video_path = 'F:\\Download July 2023\\model\\ED_sample.mp4'
cap = cv2.VideoCapture(video_path)


# In[25]:


# Get the frames per second (fps) and frame count
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))


# In[26]:


# Loop through the frames and save each frame as an image
for frame_num in range(frame_count):
    ret, frame = cap.read()
    if ret:
        frame_filename = f'frame_{frame_num:04d}.jpg'
        cv2.imwrite(frame_filename, frame)
        print(f"Saved {frame_filename}")
    else:
        print(f"End of video at frame {frame_num}")
        break

# Release the video capture object
cap.release()


# In[ ]:


  Obtaining dependency information for SpeechRecognition from https://files.pythonhosted.org/packages/73/8c/74d3b2a7d71e3f18e1e50bf3f168cf3333846137f5723efac3d0dc5a8635/SpeechRecognition-3.10.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/32.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/32.8 MB 1.9 MB/s eta 0:00:18
   ---------------------------------------- 0.1/32.8 MB 1.3 MB/s eta 0:00:26
   ---------------------------------------- 0.2/32.8 MB 1.7 MB/s eta 0:00:19
   ---------------------------------------- 0.3/32.8 MB 2.1 MB/s eta 0:00:16
    --------------------------------------- 0.5/32.8 MB 2.4 MB/s eta 0:00:14
   - -------------------------------------- 0.8/32.8 MB 3.6 MB/s eta 0:00:09
   - -------------------------------------- 1.1/32.8 MB 3.8 MB/s eta 0:00:09
   -- ------------------------------------- 1.7/32.8 MB 5.5 MB/s eta 0:00:06
   -- ------------------------------------- 2.0/32.8 MB 6.2 MB/s eta 0:00:05
   -- -------

  Obtaining dependency information for pyaudio from https://files.pythonhosted.org/packages/82/d8/f043c854aad450a76e476b0cf9cda1956419e1dacf1062eb9df3c0055abe/PyAudio-0.2.14-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/164.1 kB ? eta -:--:--
   ---------------------------------------  163.8/164.1 kB 9.6 MB/s eta 0:00:01
   ---------------------------------------- 164.1/164.1 kB 5.0 MB/s eta 0:00:00
Speak something:
Sorry, could not understand the audio
  Obtaining dependency information for gTTS from https://files.pythonhosted.org/packages/a7/ef/190f64a4edeb13165e3c598a08f06a2ae80cdae0aa208c96c20efdb7ad4b/gTTS-2.4.0-py3-none-any.whl.metadata


In [9]:
# In[2]:

#Number recognition usind MNIST dataset
import numpy as np
from keras.datasets import mnist
import matplotlib.pyplot as plt

fd
# In[3]:


#load dataset
(x_train,y_train),(x_test,y_test)=mnist.load_data()


# In[4]:


#count the unique train labels
unique, counts = np.unique(y_train, return_counts=True)
print("Train labels :", dict(zip(unique,counts)))


# In[5]:


#count the number of unique test labels
unique, count = np.unique(y_test, return_counts=True)
print("Test labels :", dict(zip(unique,counts)))


# In[6]:


#sample 25 mnist digit fromtrain dataset
indexes =np.random.randint(0, x_train.shape[0], size =25)
images=x_train[indexes]
labels=y_train[indexes]


# In[7]:


#plot the 25 mnist digits
plt.figure(figsize=(5,5))
for i in range(len(indexes)):
    plt.subplot(5,5,i+1)
    image=images[i]
    plt.imshow(image, cmap='gray')
    plt.axis('off')
plt.show()
plt.savefig("mnist-sample.png")
plt.close('all')


# In[8]:


#MNIST digit classifier model
#compute the number of labels
num_labels=len(np.unique(y_train))


# In[9]:


# convert to one hot vector
from keras.utils import to_categorical,plot_model
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)


# In[10]:


#image dimensions
image_size=x_train.shape[1]
input_size=image_size*image_size


# In[11]:


#resize and normalize
x_train= np.reshape(x_train,[-1,input_size])
x_train=x_train.astype('float32')/255
x_test =np.reshape(x_test,[-1,input_size])
x_test=x_test.astype('float32')/255


# In[13]:


#network parameter 
batch_size=128
hidden_units=256
dropout=0.45


# In[17]:


#model ia s 3- layer MLP with Relu and dropout after each layer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
model=Sequential()
model.add(Dense(hidden_units,input_dim=input_size))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(num_labels))

#this is the output for one hot encoder
model.add(Activation('softmax'))
model.summary()
plot_model(model,to_file='mlp_mnist.png',show_shapes=True)


# In[18]:


#loss function for one hot vector
#use of adam optimizer
#accuracy is a good metrics for classification tasks
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


# In[19]:


#train the network
model.fit(x_train,y_train,epochs=20,batch_size=batch_size)


# In[21]:


#validate the model on test dataset
loss,acc=model.evaluate(x_test, y_test,batch_size=batch_size)
print("\nTest accuracy:%1f%%" %(100*acc))


# In[ ]:



NameError: name 'fd' is not defined

In [10]:
# char recog using rnn cnn
# Imports 
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras import utils
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau


# In[20]:


# Startiong variables
# x_train = pd.read_csv('/kaggle/input/emnist/emnist-letters-train.csv')
x_train = pd.read_csv('C:\\Users\\DELL\\OneDrive\\Desktop\\SKIT\\Deep Learning\\Lab\\Data set\\EMNIST\\emnist-letters-train.csv')
input_shape = (28, 28, 1)


# In[21]:


x_train.head()


# In[22]:


y_train = x_train['23']
del x_train['7']


# In[23]:


# Highlighting the right answers
y_train = utils.to_categorical(y_train)


# In[24]:


# Divide the array into two parts
x_train = np.array(x_train)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)  # Convert to 2D image format
x_train = np.rot90(x_train, axes=(1, 2))
x_train = np.flip(x_train, axis=1)
x_train = x_train.astype(np.float32)


# In[25]:


# Normalize for training (from 0 to 1)
x_train /= 255.0


# In[26]:


# We divide the data into two sets, the first for training and the second for verification
    # test_size - percentage of the set for verification (10%)
    # X_train - training kit (needs to be increased)
    # X_val - verification dataset
    # Y_train - the right answers for learning
    # Y_val - correct answers to check
random_seed = 2
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=random_seed)


# In[27]:


# Expanding the data
# Create a generator that rotates, zooms, and shifts images
datagen = ImageDataGenerator(rotation_range=10, zoom_range=0.1, width_shift_range=0.1, height_shift_range=0.1)


# In[28]:


# Create a neural network
model = Sequential()
# The first block consists of two layer
# Subsample layer with selection of the maximum value in a 2 by 2# The output is a fully connected layer with 27 neurons (27 is the number of letters) squares of convolution, 
# each with 32 feature cards, the size of the convolution core is 2 by 2
model.add(Conv2D(filters=64, kernel_size=(2, 2), padding='Same', activation='relu', input_shape=input_shape))
model.add(Conv2D(filters=64, kernel_size=(2, 2), padding='Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# A layer that shuts down neurons with a 25 percent probability to prevent overlearning
model.add(Dropout(0.25))
# The second convolution block - two layers of convolution, 
# each with 32 feature cards, the size of the convolution core is 2 by 2
model.add(Conv2D(filters=64, kernel_size=(2, 2), padding='Same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(2, 2), padding='Same', activation='relu'))
# Subsample layer with selection of the maximum value in a 2 by 2 square
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# A layer that shuts down neurons with a 25 percent probability to prevent overlearning
model.add(Dropout(0.25))
# The end of the convolutional part of the neural networkmodel.add(Dense(27, activation='softmax'))


# In[29]:


# The fully connected part, responsible for classification
# The layer converts the two-dimensional output of the folding part into a one-dimensional array
model.add(Flatten())
# A fully connected layer with 256 neurons
model.add(Dense(256, activation='relu'))
# A layer that shuts down neurons with a 25 percent probability to prevent overlearning
model.add(Dropout(0.25))
# The output is a fully connected layer with 27 neurons (27 is the number of letters)
model.add(Dense(27, activation='softmax'))


# In[30]:


# Compile the neural network
# Loss function - categorical cross-entropy
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


# In[31]:


# Mini-sample size - 96 images
batch_size = 96


# In[32]:


# Network training
# We use two flasks
# The first colbel is responsible for saving the best network option
# val_acc - the proportion of correct answers on the test set, save only the best network to a file named mnist_cnn.hdf5
chekpoint = ModelCheckpoint('CNN_model_3_EMNIST_best.hdf5', monitor='val_accuracy', save_best_only=True, verbose=1)


# In[33]:


# The second callback is changing the speed of learning
# If the quality of the set under test does not change in three iterations, the speed parameter is divided in half
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5,
                                                min_lr=0.00001)


# In[ ]:


# For training, we use the generator we created earlier
history = model.fit(datagen.flow(X_train, Y_train, batch_size=batch_size), epochs=50,
                    validation_data=(X_val, Y_val), steps_per_epoch=X_train.shape[0] // batch_size, verbose=1,
                    callbacks=[chekpoint, learning_rate_reduction])


# In[ ]:


# Display the graph of the learning outcome
history_dict = history.history
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(acc_values) + 1)
plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


# In[ ]:


class_mapping = ' ABCDEFGHIJKLMNOPQRSTUVWXYZ'
import random

testing_letter = pd.read_csv('/kaggle/input/emnist/emnist-letters-test.csv')

x2 = np.array(testing_letter.iloc[:,1:].values)
test_images = x2 / 255.0

test_images_number = test_images.shape[0]
test_images_height = 28
test_images_width = 28
test_images_size = test_images_height*test_images_width

test_images = test_images.reshape(test_images_number, test_images_height, test_images_width, 1)
test_images = np.rot90(test_images, axes=(1, 2))
test_images = np.flip(test_images, axis=1)
right = 0
wrong = 0
for idx in range(1,1000):
    # idx = random.randint(0, 8000)
    result = np.argmax(model.predict(test_images[idx:idx+1]))
    answer = testing_letter.values[idx,0]
    if (result == answer):
        right += 1
    else:
        wrong += 1
print("Correct answer = ", right, " / 1 000")
print("Incorrect answer = ", wrong, " / 1 000")

    # print('Prediction: ', result, ', Char: ', class_mapping[result])
    # print('Label: ', testing_letter.values[idx,0])
    # show_img(testing_letter, idx)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\DELL\\OneDrive\\Desktop\\SKIT\\Deep Learning\\Lab\\Data set\\EMNIST\\emnist-letters-train.csv'

In [11]:
#write aprogram for character recognition using b)RNN
import numpy as np 
from keras.models import Sequential
from keras.layers import Activation ,Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.utils import to_categorical, plot_model
from keras.datasets import mnist
import pandas as pd


# In[2]:


get_ipython().system('pip install emnist')


# In[3]:


#load mnist dataset
#(x_train, y_train),(x_test, y_test)=emnist.load_data()
x=pd.read_csv('C:\\Users\\DELL\\OneDrive\\Desktop\\SKIT\\Deep Learning\\Lab\\Data set\\EMNIST\\emnist-letters-train.csv')
x1= pd.read_csv('C:\\Users\\DELL\\OneDrive\\Desktop\\SKIT\\Deep Learning\\Lab\\Data set\\EMNIST\\emnist-letters-test.csv')
                                                                                                    


# In[4]:


x.head()


# In[5]:


y_train=x['1']
x_train= x.drop(columns=['1'])
#del x_train['7']
print(x_train)
print(y_train)


# In[6]:


#compute the number of labels
num_labels=len(np.unique(y_train))
print(num_labels)


# In[103]:


x1.head()


# In[116]:


y_test=x1['1']
x_train= x1.drop(columns=['1'])
#del x_train['7']
print(x_train)
print(y_train)


# In[117]:


#compute the number of labels
num_labels=len(np.unique(y_test))
print(num_labels)


# In[126]:


num_classes = len(np.unique(y_test))
y_test = to_categorical(y_test, num_classes=num_classes+1)
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes=num_classes+1)


# In[132]:


x_train = x_train.reshape(-1, 132, 785)


# In[ ]:


model = Sequential()
model.add(SimpleRNN(128, input_shape=(sequence_length, num_features), return_sequences=True))
model.add(SimpleRNN(128, return_sequences=False))
model.add(Dense(num_classes, activation='softmax'))


# In[128]:


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# In[131]:


# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)


# In[54]:


#input image dimensions
image_size =x_train.shape[1]
#resize and normalize
x_train= np.reshape(x_train,[-1, image_size, image_size, 1])
x_test=np.reshape(x_test, [-1,image_size,image_size,1])
x_train=x_train.astype('float32')/255
x_test=x_test.astype('float32')/255


# In[82]:


#image is process as is squared grayscale
input_shape =(image_size, image_size, 1)
batch_size = 128
kernel_size = 3
pool_size =2
filters =64
dropout = 0.2
num_features=785
sequence_length=132


# In[89]:


#model is stackof CNN-ReluMaxPooling
from tensorflow.keras.layers import Dense, SimpleRNN, TimeDistributed, Flatten
# Define the RNN model
model = Sequential()
model.add(SimpleRNN(128, input_shape=(sequence_length, num_features), return_sequences=True))
model.add(SimpleRNN(128, return_sequences=False))  # You may add more layers as needed
model.add(Dense(26, activation='sigmoid'))  # Output layer for character recognition


# In[ ]:





# In[90]:


#loss function for one hot vector
# use of adam optimizer
#accuracy 

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# In[91]:


#train themodel
#model.fit(x_train, y_train, epochs=10, batch_size=batch_size)
model.fit(x_train, y_train, epochs=5, batch_size=128, validation_split=0.2)


# In[61]:


loss,acc =model.evaluate(x_test, y_test, batch_size = batch_size)
print("\n Test Accuracy :%.1f%%" % (100.0*acc))


# In[23]:


# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

# Make predictions
predictions = model.predict(x_test)


# In[38]:


import cv2
import numpy as np
from tensorflow.keras.preprocessing import image


# In[39]:


# Load and preprocess the custom image

#custom_image_path = 'C:\\Users\\DELL\\Downloads\\6.png'
custom_image_path = 'C:\\Users\\DELL\\Downloads\\MNIST_6_0.jpeg'


custom_image = image.load_img(custom_image_path, target_size=(28, 28), color_mode='grayscale')
custom_image = image.img_to_array(custom_image)
custom_image = custom_image.astype('float32') / 255.0
custom_image = np.expand_dims(custom_image, axis=0)  # Add a batch dimension


# In[40]:


# Load the image
image = mpimg.imread(custom_image_path)
# Display the image
plt.imshow(image)
#plt.axis('off')  # Turn off axis labels and ticks
plt.show()


# In[41]:


# Make predictions for the custom image
predictions = model.predict(custom_image)
#print(predictions)
# The 'predictions' variable now contains the predicted probabilities for each class (0-9).
# You can find the predicted class (character or digit) with the highest probability as follows:
predicted_class = np.argmax(predictions)

# Display the predicted class
print(f"Predicted class: {predicted_class}")


# In[28]:


import matplotlib.pyplot as plt
import matplotlib.image as mpimg


# In[30]:


# Load the image
image = mpimg.imread('C:\\Users\\DELL\\Downloads\\MNIST_6_0.jpeg')

# Display the image
plt.imshow(image)
plt.axis('off')  # Turn off axis labels and ticks
plt.show()


# In[1]:


import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


# In[ ]:


# Load the EMNIST data (assuming it's in CSV format)
# You may need to adjust the path and column names based on your dataset.
df = pd.read_csv('C:\\Users\\DELL\\OneDrive\\Desktop\\SKIT\\Deep Learning\\Lab\\Data set\\EMNIST\\emnist-letters-train.csv')


# In[ ]:


df.head()


# In[ ]:


# Preprocess the data
# Extract features (X) and labels (y) from the DataFrame
X = df.drop(columns=['1']).values
y = df['1'].values


# In[11]:


# Encode labels as integers
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)


# In[12]:


# One-hot encode the labels
num_classes = len(np.unique(y))
y = to_categorical(y, num_classes=num_classes)


# In[13]:


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# In[14]:


print(X_train.shape[1])


# In[15]:


# Define the RNN model= 784
model = Sequential()
model.add(SimpleRNN(128, input_shape=(X_train.shape[1], 1), return_sequences=True))  # Single feature per time step
model.add(SimpleRNN(128, return_sequences=False))  # You may add more layers as needed
model.add(Dense(num_classes, activation='softmax'))  # Output layer for character recognition


# In[ ]:


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)


# In[ ]:


# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")


# In[ ]:


def load_and_preprocess_image(image_path):
    # Load the image using OpenCV (assuming it's a grayscale image)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Check if the image is loaded successfully
    if image is None:
        raise ValueError(f"Failed to load image from {image_path}")
    
    # Resize the image to match the expected input size of the model
    # You may need to adjust the dimensions based on your model's input shape
    resized_image = cv2.resize(image, (28, 28))
    
    # Normalize the pixel values to the range [0, 1]
    normalized_image = resized_image / 255.0
    
    # Ensure that the image has the correct data type (float32)
    preprocessed_image = np.float32(normalized_image)
    
    return preprocessed_image


# ##### image_path = 'image_to_predict.png'  # Replace with the path to your image
# preprocessed_image = load_and_preprocess_image(image_path)

# In[ ]:


# Reshape the image data to match the model's input shape

image = image.reshape(1, sequence_length, 1)  # 1 example, sequence_length, 1 feature per pixel

# Make predictions on the image
predictions = model.predict(image)

# Get the predicted character label
predicted_label = np.argmax(predictions)

# Decode the predicted label if you've used label encoding
decoded_label = label_encoder.inverse_transform([predicted_label])[0]

print(f"Predicted Label: {decoded_label}")


^C


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\DELL\\OneDrive\\Desktop\\SKIT\\Deep Learning\\Lab\\Data set\\EMNIST\\emnist-letters-train.csv'